# Начальная инициализация

In [1]:
import datetime
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

07-March-2023 17:51:31


In [1]:
#импорт библиотек
# !pip install torch torchvision torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data_utils
# !pip install scikit-learn
from sklearn.metrics import classification_report, f1_score
import torch.optim as optim
# !pip install pandas
import pandas as pd
import numpy as np

# import re
# !pip install nltk
# import nltk
# nltk.download('punkt')
# from string import punctuation

# %pip install gensim
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

from gensim.models import FastText

In [2]:
# Подключение вычислений на видеокарту, если доступна
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
# device = "cpu"

In [3]:
print(device)

cuda:0


# Word2vec-признаки, обученные самостоятельно

## Предобработка текста

In [4]:
train = pd.read_excel('X_y_train.xlsx')
test = pd.read_excel('X_y_test.xlsx')

X_train = [el[0] for el in train[['Text']].values]
X_test = [el[0] for el in test[['Text']].values]
y_train = [el[0] for el in train[['Class']].replace(-1, 0).values]
y_test = [el[0] for el in test[['Class']].replace(-1, 0).values]

In [6]:
# Create a X_token file one time!

# def tokenize(text):

#   text_token = nltk.word_tokenize(text)
#   text_word = [el.lower() for el in text_token if el not in punctuation]
#   return text_word

# X_train_token = [tokenize(t) for t in X_train]
# X_test_token = [tokenize(t) for t in X_test]



In [7]:
# with open('X_train_token.txt', mode='w+') as file:
#     for sentence in X_train_token:
#         print(*sentence, file=file)

In [5]:
X_token = [sentence.split() for sentence in open('X_token.txt', mode='r')]
X_train_token = [sentence.split() for sentence in open('X_train_token.txt', mode='r')]
X_test_token = [sentence.split() for sentence in open('X_test_token.txt', mode='r')]

## Обучение модели word2vec на полученном наборе данных

In [6]:
w2v_model = Word2Vec(sentences=X_token, vector_size=300, window=5, min_count=1, workers=4)

In [7]:
w2v_model.build_vocab(X_token)

In [8]:
w2v_model.train(X_token, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
# w2v_model.init_sims(replace=True)

(78023432, 92066100)

In [9]:
vector = w2v_model.wv['работа']
# print(vector)
w2v_model.save('self-trained_word2vec/word2vec.model')

In [10]:
def word_averaging(model, words):
    all_words, mean = set(), []

    for word in words:
        mean.append(model.wv[word])
        all_words.add(word)

    if not mean:
        return np.zeros(model.vector_size)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

    return mean

In [11]:
def word_averaging_list(model, text_list):
    return np.vstack([word_averaging(model, comment_text) for comment_text in text_list])

In [12]:
word_vectors = w2v_model.wv
X_train = word_averaging_list(w2v_model, X_train_token)
X_test = word_averaging_list(w2v_model, X_test_token)


In [16]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

## Обучение нейронных сетей

In [13]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

NameError: name 'datetime' is not defined

In [ ]:
f1_score_w2v_self_trained = []

### Сеть c 2 скрытыми слоями, разные слои, lr=0.01

In [ ]:
def train_net(net = None, learning_rate = None, x = None, y = None, batch = None, epochs = None):
    net.to(device)

    optimizer = optim.SGD(net.parameters(), lr=learning_rate)
    
    inputs_train = torch.tensor(x).to(device)
    targets_train = torch.tensor(y).int().to(device)

    train = data_utils.TensorDataset(inputs_train, targets_train)

    trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batch)

    for epoch in range(10):
        loss = None
        for data in trainset:
            X, y = data[0].to(device), data[1].to(device)
            net.zero_grad()
            output = net(X.float())
            loss = F.cross_entropy(output, y.long())
            loss.backward()
            optimizer.step()
        print(loss)
    return net

def test_net(net = None, device = None, x = None, y = None):
    inputs_test = torch.tensor(x).to(device)
    targets_test = torch.tensor(y).int().to(device)
    
    test = data_utils.TensorDataset(inputs_test, targets_test)

    testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=1)

    ams = []
    with torch.no_grad():
        for data in testset:
            X, y = data[0].to(device), data[1].to(device)

            output = net(X.float())
            for idx, i in enumerate(output):
                ams.append(torch.argmax(i).item())
    return f1_score(y, ams, average='weighted')

In [17]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self, layers):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, layers[0])
        self.fc2 = nn.Linear(layers[0], layers[1])
        self.fc3 = nn.Linear(layers[1], layers[2])
        self.fc4 = nn.Linear(layers[2], 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [18]:
#Задаем оптимизатор и функцию потерь


# optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [19]:
#преобразование в тензоры


# inputs_train = torch.tensor(X_train).to(device)
# targets_train = torch.tensor(y_train).int().to(device)

# inputs_test = torch.tensor(X_test).to(device)
# targets_test = torch.tensor(y_test).int().to(device)

# train = data_utils.TensorDataset(inputs_train, targets_train)
# test = data_utils.TensorDataset(inputs_test, targets_test)

# trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=1)
# testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=1)

In [20]:
# for epoch in range(10): # 15 полных прохода по нашим данным
#     for data in trainset:  # `data` это батч наших данных
#         X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        
#         net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
#         output = net_2_layer(X.float())  # передаем батч
#         #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
#         loss = F.cross_entropy(output, y.long())
#         loss.backward()  # передаем это значение назад по сети
#         optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
#     print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.5990, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6365, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6091, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7265, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7582, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7801, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.2096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7568, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7670, device='cuda:0', grad_fn=<NllLossBackward0>)


In [21]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [22]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average="weighted")}')

torch.Size([1, 2])
50
              precision    recall  f1-score   support

    negative       0.77      0.74      0.76        23
    positive       0.79      0.81      0.80        27

    accuracy                           0.78        50
   macro avg       0.78      0.78      0.78        50
weighted avg       0.78      0.78      0.78        50



In [23]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.001

In [24]:
net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [25]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [26]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [28]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7005, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6971, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6874, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6893, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6969, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6970, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=

In [29]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [30]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [31]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.0001

In [32]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [33]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [34]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [35]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6831, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7053, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7392, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6809, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7090, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7055, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6860, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6878, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6990, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6884, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6963, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6961, device='cuda:0', grad_fn=

In [36]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [37]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [38]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.01

In [39]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [40]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [41]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [42]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6859, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6402, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5085, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7789, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6701, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6369, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6145, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6168, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7147, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6402, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5597, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6385, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5421, device='cuda:0', grad_fn=

In [43]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [44]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.66      0.68      0.67     33421
    positive       0.68      0.65      0.66     34396

    accuracy                           0.67     67817
   macro avg       0.67      0.67      0.67     67817
weighted avg       0.67      0.67      0.67     67817

f1 = 0.6649504364550969


In [45]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.001

In [46]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [47]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [48]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [49]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6905, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6977, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6899, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=

In [50]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [51]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.68      0.59      0.63     33421
    positive       0.65      0.72      0.68     34396

    accuracy                           0.66     67817
   macro avg       0.66      0.66      0.66     67817
weighted avg       0.66      0.66      0.66     67817

f1 = 0.6832180500658762


In [52]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.0001

In [53]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [54]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [55]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [56]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7307, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6992, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7022, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6905, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6970, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=

In [57]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [58]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [59]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.01

In [60]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [61]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [62]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [63]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6452, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5653, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6227, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6043, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5836, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6830, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5873, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5800, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4999, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6866, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5530, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6061, device='cuda:0', grad_fn=

In [64]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [65]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.65      0.70      0.67     33421
    positive       0.68      0.63      0.66     34396

    accuracy                           0.67     67817
   macro avg       0.67      0.67      0.67     67817
weighted avg       0.67      0.67      0.67     67817



In [66]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.001

In [67]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [68]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [69]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [70]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6969, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6975, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6985, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=

In [71]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [72]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.68      0.59      0.63     33421
    positive       0.65      0.73      0.69     34396

    accuracy                           0.66     67817
   macro avg       0.67      0.66      0.66     67817
weighted avg       0.67      0.66      0.66     67817



In [73]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.0001

In [74]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [75]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [76]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [77]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6858, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6852, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6867, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6993, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6897, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6899, device='cuda:0', grad_fn=

In [78]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [79]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [80]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.01

In [81]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [82]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [83]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [84]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6788, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6363, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6121, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5449, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5774, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6593, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5598, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6194, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6723, device='cuda:0', grad_fn=

In [85]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [86]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.66      0.68      0.67     33421
    positive       0.68      0.65      0.66     34396

    accuracy                           0.67     67817
   macro avg       0.67      0.67      0.67     67817
weighted avg       0.67      0.67      0.67     67817



In [87]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.001

In [88]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [89]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [90]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [91]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6980, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=

In [92]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [93]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.67      0.61      0.64     33421
    positive       0.65      0.70      0.68     34396

    accuracy                           0.66     67817
   macro avg       0.66      0.66      0.66     67817
weighted avg       0.66      0.66      0.66     67817



In [94]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.0001

In [95]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [96]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [97]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [98]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6969, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=

In [99]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [100]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [101]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.01

In [102]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [103]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [104]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [105]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6878, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7017, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6126, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6544, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5839, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6278, device='cuda:0', grad_fn=

In [106]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [107]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.64      0.73      0.68     33421
    positive       0.69      0.61      0.65     34396

    accuracy                           0.67     67817
   macro avg       0.67      0.67      0.66     67817
weighted avg       0.67      0.67      0.66     67817



In [108]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.001

In [109]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [110]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [111]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [112]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6952, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6901, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6974, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6971, device='cuda:0', grad_fn=

In [113]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [114]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.65      0.64      0.65     33421
    positive       0.66      0.67      0.66     34396

    accuracy                           0.66     67817
   macro avg       0.66      0.66      0.66     67817
weighted avg       0.66      0.66      0.66     67817



In [115]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.0001

In [116]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [117]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [118]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [119]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7200, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7062, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6889, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6879, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6976, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6924, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=

In [120]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [121]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [122]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.01

In [123]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [124]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [125]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [126]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7011, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6902, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6970, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6663, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6024, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6353, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6682, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4563, device='cuda:0', grad_fn=

In [127]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [128]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.68      0.60      0.64     33421
    positive       0.65      0.73      0.69     34396

    accuracy                           0.67     67817
   macro avg       0.67      0.67      0.66     67817
weighted avg       0.67      0.67      0.66     67817



In [129]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.001

In [130]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [131]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [132]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [133]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6870, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=

In [134]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [135]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.67      0.62      0.64     33421
    positive       0.66      0.70      0.68     34396

    accuracy                           0.66     67817
   macro avg       0.66      0.66      0.66     67817
weighted avg       0.66      0.66      0.66     67817



In [136]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.0001

In [137]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [138]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [139]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [140]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6899, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6870, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=

In [141]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [142]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [143]:
f1_score_w2v_self_trained.append(f1_score(y_test, ams, average="weighted"))

In [144]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

01-March-2023 01:42:13


# FastText-признаки, обученные самостоятельно

## Предобработка текста

In [145]:
df = pd.read_csv('df.csv', sep=';')
# df = pd.read_csv('df.csv', sep=';')

X = [el[0] for el in df[['Текст']].values]
Y = [el[0] for el in df[['Оценка']].replace(-1, 0).values]

In [146]:
# Create an X_token file one time!

# def tokenize(text):

  # without_punct = re.sub("[^А-Яа-я\sЁё]", "", text) # Удаление пунктуации (оставление только русских букв и пробелов) 
  # without_punct = re.sub('\s\s', ' ', without_punct) # Удаление двойного пробела
  # tokens = re.split(' ', without_punct) # Разделение на отдельные слова (токенизация)
  # tokens = [word for word in tokens if word.isalnum()] # Отбор именно слов и чисел!!!
  # tokens = [word.lower() for word in tokens if not word.isnumeric()] # Удаление чисел и приведение к нижнему регистру
  # return tokens

  # text_token = nltk.word_tokenize(text)
  # text_word = [el.lower() for el in text_token if el not in punctuation]
  # return text_word

# X_token = [tokenize(t) for t in X]

# with open('X_token.txt', mode='w+') as file:
#     for sentence in X_token:
#         print(*sentence, file=file)

In [147]:
X_token = [sentence.split() for sentence in open('X_token.txt', mode='r')]

## Обучение модели FastText на полученном наборе данных

In [148]:
ft_model = FastText(sentences=X_token, vector_size=300, window=5, min_count=1, workers=4)

In [149]:
ft_model.build_vocab(X_token)

In [150]:
ft_model.train(X_token, total_examples=ft_model.corpus_count, epochs=300, report_delay=1)
# ft_model.init_sims(replace=True)

(388703321, 409069200)

In [151]:
vector = ft_model.wv['работа']
# print(vector)
ft_model.save('self-trained_fasttext/fasttext.model')

In [152]:
def word_averaging(model, words):
  all_words, mean = set(), []

  for word in words:
    mean.append(ft_model.wv[word])
    all_words.add(word)

  if not mean:
    return np.zeros(model.vector_size)

  mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

  return mean

In [153]:
def word_averaging_list(model, text_list):
  return np.vstack([word_averaging(model, comment_text) for comment_text in text_list])

In [154]:
word_vectors = ft_model.wv
X = word_averaging_list(ft_model, X_token)

In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

## Обучение нейронных сетей

In [156]:
f1_score_ft_self_trained = []

### Сеть c 2 скрытыми слоями, разные слои, lr=0.01

In [157]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [158]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [159]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [160]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6969, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6456, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6335, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5727, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5758, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6276, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7338, device='cuda:0', grad_fn=

In [161]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [162]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.68      0.64      0.66     33421
    positive       0.67      0.71      0.69     34396

    accuracy                           0.68     67817
   macro avg       0.68      0.68      0.68     67817
weighted avg       0.68      0.68      0.68     67817

f1 = 0.6916644338837731


In [163]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.001

In [164]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [165]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [166]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [167]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=

In [168]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [169]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.66      0.64      0.65     33421
    positive       0.66      0.68      0.67     34396

    accuracy                           0.66     67817
   macro avg       0.66      0.66      0.66     67817
weighted avg       0.66      0.66      0.66     67817

f1 = 0.6670010173814608


In [170]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.0001

In [171]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [172]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [173]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [174]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7483, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6801, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6621, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6839, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=

In [175]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [176]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [177]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.01

In [178]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [179]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [180]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [181]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6970, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6902, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6754, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5980, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6473, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6037, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6171, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5883, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6140, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6000, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6148, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5434, device='cuda:0', grad_fn=

In [182]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [183]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.68      0.66      0.67     33421
    positive       0.68      0.69      0.68     34396

    accuracy                           0.68     67817
   macro avg       0.68      0.68      0.68     67817
weighted avg       0.68      0.68      0.68     67817

f1 = 0.6840523871821044


In [184]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.001

In [185]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [186]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [187]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [188]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6884, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6984, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6901, device='cuda:0', grad_fn=

In [189]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [190]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.67      0.66      0.66     33421
    positive       0.67      0.68      0.68     34396

    accuracy                           0.67     67817
   macro avg       0.67      0.67      0.67     67817
weighted avg       0.67      0.67      0.67     67817

f1 = 0.6754539030615924


In [191]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.0001

In [192]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [193]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [194]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [195]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6698, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6822, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6785, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6983, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6995, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6899, device='cuda:0', grad_fn=

In [196]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [197]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.62      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.57      0.50      0.34     67817
weighted avg       0.57      0.51      0.34     67817

f1 = 0.673001830085828


In [198]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.01

In [199]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [200]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [201]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [202]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6886, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6582, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5574, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6192, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6208, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6078, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.4793, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5727, device='cuda:0', grad_fn=

In [203]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [204]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.67      0.69      0.68     33421
    positive       0.69      0.67      0.68     34396

    accuracy                           0.68     67817
   macro avg       0.68      0.68      0.68     67817
weighted avg       0.68      0.68      0.68     67817



In [205]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.001

In [206]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [207]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [208]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [209]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6879, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6952, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=

In [210]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [211]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [212]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.0001

In [213]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [214]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [215]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [216]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7099, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7319, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6729, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7163, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=

In [217]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [218]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [219]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.01

In [220]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [221]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [222]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [223]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6977, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6873, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6018, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6770, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6457, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6099, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6156, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5691, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7400, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7128, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5281, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6124, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5642, device='cuda:0', grad_fn=

In [224]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [225]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.68      0.65      0.66     33421
    positive       0.67      0.71      0.69     34396

    accuracy                           0.68     67817
   macro avg       0.68      0.68      0.68     67817
weighted avg       0.68      0.68      0.68     67817



In [226]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.001

In [227]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [228]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [229]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [230]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6960, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6899, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6905, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6888, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6868, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6952, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=

In [231]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [232]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.68      0.61      0.64     33421
    positive       0.65      0.72      0.69     34396

    accuracy                           0.67     67817
   macro avg       0.67      0.67      0.66     67817
weighted avg       0.67      0.67      0.66     67817



In [233]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.0001

In [234]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [235]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [236]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [237]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6744, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6980, device='cuda:0', grad_fn=

In [238]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [239]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [240]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.01

In [241]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [242]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [243]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [244]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6805, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6968, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6968, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6871, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6063, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6217, device='cuda:0', grad_fn=

In [245]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [246]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.71      0.57      0.63     33421
    positive       0.65      0.77      0.71     34396

    accuracy                           0.68     67817
   macro avg       0.68      0.67      0.67     67817
weighted avg       0.68      0.68      0.67     67817



In [247]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.001

In [248]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [249]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [250]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [251]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6977, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6887, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6874, device='cuda:0', grad_fn=

In [252]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [253]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [254]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.0001

In [255]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [256]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [257]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [258]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6883, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=

In [259]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [260]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [261]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.01

In [262]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [263]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [264]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [265]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6924, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6891, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6880, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6801, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5785, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6334, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5402, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6010, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5368, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6128, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6322, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6759, device='cuda:0', grad_fn=

In [266]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [267]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.72      0.54      0.62     33421
    positive       0.64      0.80      0.71     34396

    accuracy                           0.67     67817
   macro avg       0.68      0.67      0.67     67817
weighted avg       0.68      0.67      0.67     67817



In [268]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.001

In [269]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [270]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [271]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [272]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6871, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6884, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6976, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=

In [273]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [274]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [275]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.0001

In [276]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [277]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [278]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [279]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6888, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=

In [280]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [281]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [282]:
f1_score_ft_self_trained.append(f1_score(y_test, ams, average="weighted"))

In [283]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

01-March-2023 04:59:46


# Word2Vec-признаки предобученные

## Предобработка текста

In [284]:
df = pd.read_csv('df.csv', sep=';')

X = [el[0] for el in df[['Текст']].values]
Y = [el[0] for el in df[['Оценка']].replace(-1, 0).values]

In [285]:
import sys
import os
!pip install wget
import wget
import re
!pip install ufal.udpipe
from ufal.udpipe import Model, Pipeline

"""
Этот скрипт принимает на вход необработанный русский текст 
(одно предложение на строку или один абзац на строку).
Он токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи 
("зеленый_ADJ трамвай_NOUN").
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).

Примеры запуска:
echo 'Мама мыла раму.' | python3 rus_preprocessing_udpipe.py
zcat large_corpus.txt.gz | python3 rus_preprocessing_udpipe.py | gzip > processed_corpus.txt.gz
"""


def num_replace(word):
    newtoken = "x" * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(" ", "")
    if token == "Файл" and "SpaceAfter=No" in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(" ", "").replace("_", "").lower()
    if "|" in out_lemma or out_lemma.endswith(".jpg") or out_lemma.endswith(".png"):
        return None
    if pos != "PUNCT":
        if out_lemma.startswith("«") or out_lemma.startswith("»"):
            out_lemma = "".join(out_lemma[1:])
        if out_lemma.endswith("«") or out_lemma.endswith("»"):
            out_lemma = "".join(out_lemma[:-1])
        if (
            out_lemma.endswith("!")
            or out_lemma.endswith("?")
            or out_lemma.endswith(",")
            or out_lemma.endswith(".")
        ):
            out_lemma = "".join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace(
        "\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019",
        "\u0022",
        text,
    )

    text = list_replace(
        "\u2012\u2013\u2014\u2015\u203E\u0305\u00AF", "\u2003\u002D\u002D\u2003", text
    )

    text = list_replace("\u2010\u2011", "\u002D", text)

    text = list_replace(
        "\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000",
        "\u2002",
        text,
    )

    text = re.sub("\u2003\u2003", "\u2003", text)
    text = re.sub("\t\t", "\t", text)

    text = list_replace(
        "\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062",
        ".",
        text,
    )

    text = list_replace("\u2217", "\u002A", text)

    text = list_replace("…", "...", text)

    text = list_replace("\u2241\u224B\u2E2F\u0483", "\u223D", text)

    text = list_replace("\u00C4", "A", text)  # латинская
    text = list_replace("\u00E4", "a", text)
    text = list_replace("\u00CB", "E", text)
    text = list_replace("\u00EB", "e", text)
    text = list_replace("\u1E26", "H", text)
    text = list_replace("\u1E27", "h", text)
    text = list_replace("\u00CF", "I", text)
    text = list_replace("\u00EF", "i", text)
    text = list_replace("\u00D6", "O", text)
    text = list_replace("\u00F6", "o", text)
    text = list_replace("\u00DC", "U", text)
    text = list_replace("\u00FC", "u", text)
    text = list_replace("\u0178", "Y", text)
    text = list_replace("\u00FF", "y", text)
    text = list_replace("\u00DF", "s", text)
    text = list_replace("\u1E9E", "S", text)

    currencies = list(
        "\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192"
    )

    alphabet = list(
        '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '
    )

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = "".join(cleaned_text)

    return cleaned_text


def process(pipeline, text="Строка", keep_pos=True, keep_punct=False):
    # Если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы
    # По умолчанию знаки пунктуации вырезаются. Чтобы сохранить их, выставьте punct=True

    entities = {"PROPN"}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [line for line in processed.split("\n") if not line.startswith("#")]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split("\t") for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if "|" not in feats:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            morph = {el.split("=")[0]: el.split("=")[1] for el in feats.split("|")}
            if "Case" not in morph or "Number" not in morph:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph["Case"]
                mem_number = morph["Number"]
            if morph["Case"] == mem_case and morph["Number"] == mem_number:
                memory.append(lemma)
                if "SpacesAfter=\\n" in misc or "SpacesAfter=\s\\n" in misc:
                    named = False
                    past_lemma = "::".join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + "_PROPN")
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))
        else:
            if not named:
                if (
                    pos == "NUM" and token.isdigit()
                ):  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append("%s_%s" % (lemma, pos))
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split("_")[1] != "PUNCT"]
    if not keep_pos:
        tagged_propn = [word.split("_")[0] for word in tagged_propn]
    return tagged_propn


# URL of the UDPipe model
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = '' + udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

print("\nLoading the model...", file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)

print("Processing input...", file=sys.stderr)
X_token = []
for input_line in df['Текст']:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res)
    X_token.append(output)


Loading the model...
Processing input...


## Обучение модели Word2Vec на полученном наборе данных

In [286]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec/model.bin', encoding='utf-8', unicode_errors='ignore', binary=True)
w2v_model.fill_norms(force=True)

In [287]:
vector = w2v_model[w2v_model.key_to_index['работа_NOUN']]
print(vector)
# w2v_model.save('fasttext.model')

[-3.02568102e+00  4.10510159e+00  3.96478510e+00  1.99146974e+00
  2.75750041e-01  1.05907798e+00 -2.34842032e-01  1.13069057e+00
 -3.34521389e+00  5.44947767e+00 -1.78760219e+00  2.93239641e+00
  4.24604845e+00 -3.08241105e+00 -3.06380081e+00  4.14739418e+00
  1.94640350e+00 -6.41723156e+00  4.48100597e-01  1.98949501e-01
 -1.96533740e+00  2.04884505e+00  6.81714356e-01  1.89991868e+00
 -1.87503803e+00  1.61289966e+00  8.80351245e-01  1.27756655e+00
 -1.60905108e-01 -2.56419826e+00  5.59642196e-01 -2.38538122e+00
  5.61529458e-01 -3.78697932e-01 -5.17279339e+00 -4.75222081e-01
 -7.38774776e-01 -1.47579443e+00  5.36987162e+00  1.66592920e+00
  2.48067904e+00 -3.41140532e+00 -3.26146185e-01  1.99449539e+00
  1.23095262e+00  1.58154178e+00  1.74632573e+00  1.36116230e+00
 -2.66357565e+00  6.55228496e-01  2.66324496e+00 -2.19835329e+00
 -7.47147948e-02  2.37323642e+00 -2.46333432e+00  2.09079415e-01
 -2.15091515e+00 -5.49663115e+00 -8.21452737e-01  1.57285839e-01
  6.61701798e-01 -3.56726

In [288]:
def word_averaging(model, words):
  all_words, mean = set(), []

  for word in words:
    if word in model.key_to_index:
      mean.append(model[model.key_to_index[word]])
      all_words.add(word)

  if not mean:
    return np.zeros(model.vector_size)

  mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

  return mean

In [289]:
def word_averaging_list(model, text_list):
  return np.vstack([word_averaging(model, comment_text) for comment_text in text_list])

In [290]:
# word_vectors = w2v_model[:]
X = word_averaging_list(w2v_model, X_token)

In [291]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

## Обучение нейронных сетей

In [292]:
f1_score_w2v_pre_trained = []

### Сеть c 2 скрытыми слоями, разные слои, lr=0.01

In [293]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [294]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [295]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [296]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6952, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7026, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6777, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6594, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6529, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6680, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7241, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6588, device='cuda:0', grad_fn=

In [297]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [298]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.57      0.76      0.65     33421
    positive       0.65      0.45      0.53     34396

    accuracy                           0.60     67817
   macro avg       0.61      0.60      0.59     67817
weighted avg       0.61      0.60      0.59     67817

f1 = 0.5311814965047036


In [299]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.001

In [300]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [301]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [302]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [303]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6996, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6971, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6901, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=

In [304]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [305]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [306]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.0001

In [307]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [308]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [309]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [310]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7260, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7150, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7540, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6812, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7121, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7031, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6813, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6873, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6957, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6886, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6960, device='cuda:0', grad_fn=

In [311]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [312]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [313]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.01

In [314]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [315]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [316]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [317]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6329, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6824, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6476, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6512, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6613, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6569, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6719, device='cuda:0', grad_fn=

In [318]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [319]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.57      0.76      0.65     33421
    positive       0.65      0.44      0.52     34396

    accuracy                           0.60     67817
   macro avg       0.61      0.60      0.59     67817
weighted avg       0.61      0.60      0.59     67817

f1 = 0.5246670255439596


In [320]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.001

In [321]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [322]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [323]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [324]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6886, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6924, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6987, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=

In [325]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [326]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.60      0.51      0.55     33421
    positive       0.58      0.67      0.62     34396

    accuracy                           0.59     67817
   macro avg       0.59      0.59      0.58     67817
weighted avg       0.59      0.59      0.58     67817

f1 = 0.6211864751046053


In [327]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.0001

In [328]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [329]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [330]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [331]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7051, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7150, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6975, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=

In [332]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [333]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [334]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.01

In [335]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [336]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [337]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [338]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6985, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6818, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6409, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6373, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6331, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6713, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6681, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6559, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6563, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6641, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6497, device='cuda:0', grad_fn=

In [339]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [340]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.59      0.64      0.61     33421
    positive       0.62      0.58      0.60     34396

    accuracy                           0.61     67817
   macro avg       0.61      0.61      0.61     67817
weighted avg       0.61      0.61      0.61     67817



In [341]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.001

In [342]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [343]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [344]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [345]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6923, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6852, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6957, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6854, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6865, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=

In [346]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [347]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.58      0.66      0.62     33421
    positive       0.62      0.53      0.57     34396

    accuracy                           0.59     67817
   macro avg       0.60      0.60      0.59     67817
weighted avg       0.60      0.59      0.59     67817



In [348]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.0001

In [349]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [350]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [351]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [352]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6968, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6901, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6957, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6969, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=

In [353]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [354]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [355]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.01

In [356]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [357]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [358]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [359]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6845, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6534, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6721, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6356, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6158, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6092, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6640, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6659, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.5453, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6653, device='cuda:0', grad_fn=

In [360]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [361]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.59      0.67      0.63     33421
    positive       0.63      0.54      0.58     34396

    accuracy                           0.61     67817
   macro avg       0.61      0.61      0.60     67817
weighted avg       0.61      0.61      0.60     67817



In [362]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.001

In [363]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [364]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [365]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [366]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6905, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6973, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=

In [367]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [368]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.61      0.50      0.55     33421
    positive       0.58      0.68      0.63     34396

    accuracy                           0.59     67817
   macro avg       0.60      0.59      0.59     67817
weighted avg       0.59      0.59      0.59     67817



In [369]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.0001

In [370]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [371]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [372]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [373]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6836, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6963, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6860, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=

In [374]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [375]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [376]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.01

In [377]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [378]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [379]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [380]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6971, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6887, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=

In [381]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [382]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.59      0.67      0.63     33421
    positive       0.63      0.55      0.59     34396

    accuracy                           0.61     67817
   macro avg       0.61      0.61      0.61     67817
weighted avg       0.61      0.61      0.61     67817



In [383]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.001

In [384]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [385]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [386]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [387]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6896, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6902, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6977, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6947, device='cuda:0', grad_fn=

In [388]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [389]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [390]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.0001

In [391]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [392]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [393]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [394]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7088, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6996, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=

In [395]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [396]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [397]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.01

In [398]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [399]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [400]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [401]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6870, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6879, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6899, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=

In [402]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [403]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.63      0.50      0.56     33421
    positive       0.59      0.71      0.65     34396

    accuracy                           0.61     67817
   macro avg       0.61      0.60      0.60     67817
weighted avg       0.61      0.61      0.60     67817



In [404]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.001

In [405]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [406]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [407]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [408]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6985, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6970, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6905, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6985, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6878, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=

In [409]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [410]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [411]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.0001

In [412]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [413]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [414]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [415]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7217, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6905, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6891, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6924, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6889, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6952, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=

In [416]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [417]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [418]:
f1_score_w2v_pre_trained.append(f1_score(y_test, ams, average="weighted"))

In [419]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

01-March-2023 08:05:08


# FastText-признаки предобученные

## Предобработка текста

In [420]:
df = pd.read_csv('df.csv', sep=';')

X = [el[0] for el in df[['Текст']].values]
Y = [el[0] for el in df[['Оценка']].replace(-1, 0).values]

In [421]:
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline

"""
Этот скрипт принимает на вход необработанный русский текст 
(одно предложение на строку или один абзац на строку).
Он токенизируется, лемматизируется и размечается по частям речи с использованием UDPipe.
На выход подаётся последовательность разделенных пробелами лемм с частями речи 
("зеленый_ADJ трамвай_NOUN").
Их можно непосредственно использовать в моделях с RusVectōrēs (https://rusvectores.org).

Примеры запуска:
echo 'Мама мыла раму.' | python3 rus_preprocessing_udpipe.py
zcat large_corpus.txt.gz | python3 rus_preprocessing_udpipe.py | gzip > processed_corpus.txt.gz
"""


def num_replace(word):
    newtoken = "x" * len(word)
    return newtoken


def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(" ", "")
    if token == "Файл" and "SpaceAfter=No" in misc:
        return None
    return out_token


def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(" ", "").replace("_", "").lower()
    if "|" in out_lemma or out_lemma.endswith(".jpg") or out_lemma.endswith(".png"):
        return None
    if pos != "PUNCT":
        if out_lemma.startswith("«") or out_lemma.startswith("»"):
            out_lemma = "".join(out_lemma[1:])
        if out_lemma.endswith("«") or out_lemma.endswith("»"):
            out_lemma = "".join(out_lemma[:-1])
        if (
            out_lemma.endswith("!")
            or out_lemma.endswith("?")
            or out_lemma.endswith(",")
            or out_lemma.endswith(".")
        ):
            out_lemma = "".join(out_lemma[:-1])
    return out_lemma


def list_replace(search, replacement, text):
    search = [el for el in search if el in text]
    for c in search:
        text = text.replace(c, replacement)
    return text


def unify_sym(text):  # принимает строку в юникоде
    text = list_replace(
        "\u00AB\u00BB\u2039\u203A\u201E\u201A\u201C\u201F\u2018\u201B\u201D\u2019",
        "\u0022",
        text,
    )

    text = list_replace(
        "\u2012\u2013\u2014\u2015\u203E\u0305\u00AF", "\u2003\u002D\u002D\u2003", text
    )

    text = list_replace("\u2010\u2011", "\u002D", text)

    text = list_replace(
        "\u2000\u2001\u2002\u2004\u2005\u2006\u2007\u2008\u2009\u200A\u200B\u202F\u205F\u2060\u3000",
        "\u2002",
        text,
    )

    text = re.sub("\u2003\u2003", "\u2003", text)
    text = re.sub("\t\t", "\t", text)

    text = list_replace(
        "\u02CC\u0307\u0323\u2022\u2023\u2043\u204C\u204D\u2219\u25E6\u00B7\u00D7\u22C5\u2219\u2062",
        ".",
        text,
    )

    text = list_replace("\u2217", "\u002A", text)

    text = list_replace("…", "...", text)

    text = list_replace("\u2241\u224B\u2E2F\u0483", "\u223D", text)

    text = list_replace("\u00C4", "A", text)  # латинская
    text = list_replace("\u00E4", "a", text)
    text = list_replace("\u00CB", "E", text)
    text = list_replace("\u00EB", "e", text)
    text = list_replace("\u1E26", "H", text)
    text = list_replace("\u1E27", "h", text)
    text = list_replace("\u00CF", "I", text)
    text = list_replace("\u00EF", "i", text)
    text = list_replace("\u00D6", "O", text)
    text = list_replace("\u00F6", "o", text)
    text = list_replace("\u00DC", "U", text)
    text = list_replace("\u00FC", "u", text)
    text = list_replace("\u0178", "Y", text)
    text = list_replace("\u00FF", "y", text)
    text = list_replace("\u00DF", "s", text)
    text = list_replace("\u1E9E", "S", text)

    currencies = list(
        "\u20BD\u0024\u00A3\u20A4\u20AC\u20AA\u2133\u20BE\u00A2\u058F\u0BF9\u20BC\u20A1\u20A0\u20B4\u20A7\u20B0\u20BF\u20A3\u060B\u0E3F\u20A9\u20B4\u20B2\u0192\u20AB\u00A5\u20AD\u20A1\u20BA\u20A6\u20B1\uFDFC\u17DB\u20B9\u20A8\u20B5\u09F3\u20B8\u20AE\u0192"
    )

    alphabet = list(
        '\t\n\r абвгдеёзжийклмнопрстуфхцчшщьыъэюяАБВГДЕЁЗЖИЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ,.[]{}()=+-−*&^%$#@!?~;:0123456789§/\|"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ '
    )

    alphabet.append("'")

    allowed = set(currencies + alphabet)

    cleaned_text = [sym for sym in text if sym in allowed]
    cleaned_text = "".join(cleaned_text)

    return cleaned_text


def process(pipeline, text="Строка", keep_pos=True, keep_punct=False):
    # Если частеречные тэги не нужны (например, их нет в модели), выставьте pos=False
    # в этом случае на выход будут поданы только леммы
    # По умолчанию знаки пунктуации вырезаются. Чтобы сохранить их, выставьте punct=True

    entities = {"PROPN"}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []

    # обрабатываем текст, получаем результат в формате conllu:
    processed = pipeline.process(text)

    # пропускаем строки со служебной информацией:
    content = [line for line in processed.split("\n") if not line.startswith("#")]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split("\t") for w in content if w]

    for t in tagged:
        if len(t) != 10:
            continue
        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t
        token = clean_token(token, misc)
        lemma = clean_lemma(lemma, pos)
        if not lemma or not token:
            continue
        if pos in entities:
            if "|" not in feats:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            morph = {el.split("=")[0]: el.split("=")[1] for el in feats.split("|")}
            if "Case" not in morph or "Number" not in morph:
                tagged_propn.append("%s_%s" % (lemma, pos))
                continue
            if not named:
                named = True
                mem_case = morph["Case"]
                mem_number = morph["Number"]
            if morph["Case"] == mem_case and morph["Number"] == mem_number:
                memory.append(lemma)
                if "SpacesAfter=\\n" in misc or "SpacesAfter=\s\\n" in misc:
                    named = False
                    past_lemma = "::".join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + "_PROPN")
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))
        else:
            if not named:
                if (
                    pos == "NUM" and token.isdigit()
                ):  # Заменяем числа на xxxxx той же длины
                    lemma = num_replace(token)
                tagged_propn.append("%s_%s" % (lemma, pos))
            else:
                named = False
                past_lemma = "::".join(memory)
                memory = []
                tagged_propn.append(past_lemma + "_PROPN")
                tagged_propn.append("%s_%s" % (lemma, pos))

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split("_")[1] != "PUNCT"]
    if not keep_pos:
        tagged_propn = [word.split("_")[0] for word in tagged_propn]
    return tagged_propn


# URL of the UDPipe model
udpipe_model_url = "https://rusvectores.org/static/models/udpipe_syntagrus.model"
udpipe_filename = '' + udpipe_model_url.split("/")[-1]

if not os.path.isfile(udpipe_filename):
    print("UDPipe model not found. Downloading...", file=sys.stderr)
    wget.download(udpipe_model_url)

print("\nLoading the model...", file=sys.stderr)
model = Model.load(udpipe_filename)
process_pipeline = Pipeline(
    model, "tokenize", Pipeline.DEFAULT, Pipeline.DEFAULT, "conllu"
)

print("Processing input...", file=sys.stderr)
X_token = []
for input_line in df['Текст']:
    res = unify_sym(input_line.strip())
    output = process(process_pipeline, text=res, keep_pos=False)
    X_token.append(output)


Loading the model...
Processing input...


## Обучение модели FastText на полученном наборе данных

In [422]:
ft_model = gensim.models.KeyedVectors.load('fasttext/model.model')
ft_model.fill_norms(force=True)

In [423]:
vector = ft_model[ft_model.key_to_index['работа']]
print(vector)
# w2v_model.save('fasttext.model')

[-1.28143653e-01  6.31993353e-01  3.97833914e-01 -1.69036105e-01
  1.94467843e-01  2.62616009e-01 -1.29946560e-01  2.99574584e-01
  5.06506674e-02  5.92105603e-03  8.87992755e-02  4.69435602e-01
 -2.00550243e-01  6.64121136e-02  1.52241021e-01 -3.72883081e-01
 -3.08863670e-02 -1.04671396e-01  1.48081467e-01 -8.00064430e-02
  7.97154009e-02 -1.13625549e-01 -3.49553585e-01 -1.42057359e-01
  7.67205238e-01 -1.68227687e-01 -1.12291731e-01  3.17529649e-01
 -3.59128028e-01 -6.41788542e-02 -5.57220131e-02  1.65108591e-01
  1.66394070e-01  5.43315411e-01 -1.59994856e-01  2.17812255e-01
  2.80040950e-01 -5.70807010e-02  1.74972832e-01  1.98939666e-01
  1.93150759e-01  2.96664566e-01  7.72891268e-02  5.42501509e-01
  2.76810322e-02 -1.13098420e-01  1.08400442e-01  6.68987632e-03
 -1.36405602e-01 -9.27114263e-02 -5.01722358e-02 -3.12990457e-01
  7.97539577e-02  3.24938953e-01 -1.21801049e-01  2.70672590e-01
 -4.02754359e-02  3.37236971e-01 -4.76787239e-01 -1.47199020e-01
  8.80351439e-02 -2.75021

In [424]:
def word_averaging(model, words):
  all_words, mean = set(), []

  for word in words:
    if word in model.key_to_index:
      mean.append(model[model.key_to_index[word]])
      all_words.add(word)

  if not mean:
    return np.zeros(model.vector_size)

  mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)

  return mean

In [425]:
def word_averaging_list(model, text_list):
  return np.vstack([word_averaging(model, comment_text) for comment_text in text_list])

In [426]:
# word_vectors = w2v_model[:]
X = word_averaging_list(ft_model, X_token)

In [427]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

## Обучение нейронных сетей

In [428]:
f1_score_ft_pre_trained = []

### Сеть c 2 скрытыми слоями, разные слои, lr=0.01

In [429]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [430]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [431]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [432]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6995, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6934, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6924, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6949, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6992, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6905, device='cuda:0', grad_fn=

In [433]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [434]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.61      0.51      0.56     33421
    positive       0.59      0.68      0.63     34396

    accuracy                           0.60     67817
   macro avg       0.60      0.60      0.59     67817
weighted avg       0.60      0.60      0.60     67817

f1 = 0.6313555312216951


In [435]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.001

In [436]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [437]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [438]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [439]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6970, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6952, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6915, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6966, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6912, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6902, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6952, device='cuda:0', grad_fn=

In [440]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [441]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [442]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, разные слои, lr=0.0001

In [443]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=4, bias=True)
  (fc4): Linear(in_features=4, out_features=2, bias=True)
)


In [444]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [445]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [446]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7234, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7113, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6764, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7022, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6858, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6983, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=

In [447]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [448]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [449]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.01

In [450]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [451]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [452]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [453]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6992, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6924, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6967, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6964, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6947, device='cuda:0', grad_fn=

In [454]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [455]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.57      0.71      0.64     33421
    positive       0.64      0.49      0.55     34396

    accuracy                           0.60     67817
   macro avg       0.61      0.60      0.59     67817
weighted avg       0.61      0.60      0.59     67817

f1 = 0.5507251159030538


In [456]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.001

In [457]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [458]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [459]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [460]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6891, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=

In [461]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [462]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.55      0.06      0.10     33421
    positive       0.51      0.95      0.66     34396

    accuracy                           0.51     67817
   macro avg       0.53      0.51      0.38     67817
weighted avg       0.53      0.51      0.39     67817

f1 = 0.6646534864160999


In [463]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 2 скрытыми слоями, одинаковые слои, lr=0.0001

In [464]:
#Архитектура нейронной сети
class Net_2_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 16)
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net_2_layer = Net_2_layer().to(device)
print(net_2_layer)

Net_2_layer(
  (fc1): Linear(in_features=300, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=2, bias=True)
)


In [465]:
#Задаем оптимизатор и функцию потерь


optimizer_2_layer = optim.SGD(net_2_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [466]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [467]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_2_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_2_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_2_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7096, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6866, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6877, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6963, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=

In [468]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_2_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [469]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
print(f'f1 = {f1_score(y_test, ams)}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817

f1 = 0.6730259360355336


/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [470]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.01

In [471]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [472]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [473]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [474]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6941, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7007, device='cuda:0', grad_fn=

In [475]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [476]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.63      0.37      0.47     33421
    positive       0.56      0.79      0.66     34396

    accuracy                           0.58     67817
   macro avg       0.60      0.58      0.56     67817
weighted avg       0.60      0.58      0.56     67817



In [477]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.001

In [478]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [479]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [480]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [481]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7036, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6876, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6943, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=

In [482]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [483]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [484]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, разные слои, lr=0.0001

In [485]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 4)
        self.fc5 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
)


In [486]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [487]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [488]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6804, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6980, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7268, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6981, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6894, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6847, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6884, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6825, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6902, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6882, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6884, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=

In [489]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [490]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [491]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.01

In [492]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [493]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [494]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [495]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7009, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6975, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6972, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6933, device='cuda:0', grad_fn=

In [496]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [497]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.59      0.69      0.63     33421
    positive       0.63      0.53      0.58     34396

    accuracy                           0.61     67817
   macro avg       0.61      0.61      0.60     67817
weighted avg       0.61      0.61      0.60     67817



In [498]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.001

In [499]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [500]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [501]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [502]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6922, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6907, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6889, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6901, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6893, device='cuda:0', grad_fn=

In [503]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [504]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [505]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 3 скрытыми слоями, одинаковые слои, lr=0.0001

In [506]:
#Архитектура нейронной сети
class Net_3_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc5 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return F.log_softmax(x, dim=1)

net_3_layer = Net_3_layer().to(device)
print(net_3_layer)

Net_3_layer(
  (fc1): Linear(in_features=300, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=2, bias=True)
)


In [507]:
#Задаем оптимизатор и функцию потерь


optimizer_3_layer = optim.SGD(net_3_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [508]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [509]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_3_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_3_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_3_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.7143, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6931, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6739, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7028, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6875, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6867, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6906, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6917, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6945, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=

In [510]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_3_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [511]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [512]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.01

In [513]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [514]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [515]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [516]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6932, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6921, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6962, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6879, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6947, device='cuda:0', grad_fn=

In [517]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [518]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [519]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.001

In [520]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [521]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [522]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [523]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6882, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6909, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6964, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6971, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6892, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6981, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6991, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6949, device='cuda:0', grad_fn=

In [524]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [525]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [526]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, разные слои, lr=0.0001

In [527]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, 4)
        self.fc6 = nn.Linear(4, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=8, bias=True)
  (fc5): Linear(in_features=8, out_features=4, bias=True)
  (fc6): Linear(in_features=4, out_features=2, bias=True)
)


In [528]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [529]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [530]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6815, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7016, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6903, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6964, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6926, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7013, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6884, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6938, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6946, device='cuda:0', grad_fn=

In [531]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [532]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [533]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.01

In [534]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [535]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.01) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [536]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [537]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6968, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6940, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6916, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6914, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6954, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6877, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6981, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6920, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6968, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6930, device='cuda:0', grad_fn=

In [538]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [539]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.58      0.67      0.63     33421
    positive       0.63      0.54      0.58     34396

    accuracy                           0.60     67817
   macro avg       0.61      0.60      0.60     67817
weighted avg       0.61      0.60      0.60     67817



In [540]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.001

In [541]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [542]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [543]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [544]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6890, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6928, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6911, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6976, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6936, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6939, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6929, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6958, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6950, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6960, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6918, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6951, device='cuda:0', grad_fn=

In [545]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [546]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [547]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

### Сеть c 4 скрытыми слоями, одинаковые слои, lr=0.0001

In [548]:
#Архитектура нейронной сети
class Net_4_layer(nn.Module):
    def __init__(self):
        super().__init__()  #вх. #вых.
        self.fc1 = nn.Linear(300, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 64)
        self.fc6 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return F.log_softmax(x, dim=1)

net_4_layer = Net_4_layer().to(device)
print(net_4_layer)

Net_4_layer(
  (fc1): Linear(in_features=300, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=64, bias=True)
  (fc6): Linear(in_features=64, out_features=2, bias=True)
)


In [549]:
#Задаем оптимизатор и функцию потерь


optimizer_4_layer = optim.SGD(net_4_layer.parameters(), lr=0.0001) # регулирует изменяемые параметры нашей модели, а именно веса, чтобы они постепенно, с течением времени, приходили в соответствие нашим данным
# lr - скорость обучения определяет величину изменений, которую оптимайзер может внести за один раз
# Каждый полный проход по данным называется эпохой (epoch)

In [550]:
#преобразование в тензоры
inputs_train = torch.tensor(X_train).to(device)
targets_train = torch.tensor(y_train).int().to(device)

inputs_test = torch.tensor(X_test).to(device)
targets_test = torch.tensor(y_test).int().to(device)

train = data_utils.TensorDataset(inputs_train, targets_train)
test = data_utils.TensorDataset(inputs_test, targets_test)

trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=32)

In [551]:
for epoch in range(100): # 15 полных прохода по нашим данным
    for data in trainset:  # `data` это батч наших данных
        X, y = data[0].to(device), data[1].to(device)  # X это батч свойств, y это батч целевых переменных.
        net_4_layer.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь. Вам следует делать это на каждом шаге.
        output = net_4_layer(X.float())  # передаем батч
        #loss = F.nll_loss(output, y.long())  # вычисляем функцию потерь
        loss = F.cross_entropy(output, y.long())
        loss.backward()  # передаем это значение назад по сети
        optimizer_4_layer.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента
    print(loss)  # выводим на экран значение функции потерь. Мы надеемся, что оно убывает!

tensor(0.6959, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6802, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.7008, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6994, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6956, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6904, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6895, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6927, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6955, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6919, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.6963, device='cuda:0', grad_fn=

In [552]:
#Предсказание меток классов для примеров из тестового множества
ams = []
with torch.no_grad():
    for data in testset:
        X, y = data[0].to(device), data[1].to(device)
        output = net_4_layer(X.float())
        for idx, i in enumerate(output):
            ams.append(torch.argmax(i).item())

In [553]:
#Оценка качества классификации
print(output.size())
print(len(ams))

target_names = ['negative', 'positive']
print(classification_report(y_test, ams, target_names=target_names))
# print(f'f1 = {f1_score(y_test, ams, average='weighted', labels = [0, 1])}')

torch.Size([9, 2])
67817
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00     33421
    positive       0.51      1.00      0.67     34396

    accuracy                           0.51     67817
   macro avg       0.25      0.50      0.34     67817
weighted avg       0.26      0.51      0.34     67817



/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ext-shirokov-m@ad.speechpro.com/shirokov/jupyter/env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

In [554]:
f1_score_ft_pre_trained.append(f1_score(y_test, ams, average="weighted"))

In [555]:
print(datetime.datetime.now().strftime('%d-%B-%Y %H:%M:%S'))

01-March-2023 11:11:22


# Сводная таблица

In [572]:
summary = pd.DataFrame()
summary['Количество скрытых слоев'] = np.array([2] * 6 + [3] * 6 + [4] * 6, dtype=int)
summary['Количество нейронов'] = np.array(['16-8'] * 3 + ['16-16'] * 3 + ['32-16-8'] * 3 + ['32-32-32'] * 3 + ['64-32-16-8'] * 3 + ['64-64-64-64'] * 3, dtype=str)
summary['Скорость обучения'] = np.array(['0.01', '0.001', '0.0001'] * 6, dtype=str)
summary['Weighted F1-score self-trained W2V'] = np.array([round(el, 2) for el in f1_score_w2v_self_trained], dtype=float)
summary['Weighted F1-score self-trained FT'] = np.array([round(el, 2) for el in f1_score_ft_self_trained], dtype=float)
summary['Weighted F1-score pre-trained W2V'] = np.array([round(el, 2) for el in f1_score_w2v_pre_trained], dtype=float)
summary['Weighted F1-score pre-trained FT'] = np.array([round(el, 2) for el in f1_score_ft_pre_trained], dtype=float)
summary

,Количество скрытых слоев,Количество нейронов,Скорость обучения,Weighted F1-score self-trained W2V,Weighted F1-score self-trained FT,Weighted F1-score pre-trained W2V,Weighted F1-score pre-trained FT
0,2,16-8,0.01,0.66,0.68,0.59,0.60
1,2,16-8,0.001,0.34,0.66,0.34,0.34
2,2,16-8,0.0001,0.34,0.34,0.34,0.34
3,2,16-16,0.01,0.67,0.68,0.59,0.59
4,2,16-16,0.001,0.66,0.67,0.58,0.39
5,2,16-16,0.0001,0.34,0.34,0.34,0.34
6,3,32-16-8,0.01,0.67,0.68,0.61,0.56
7,3,32-16-8,0.001,0.66,0.34,0.59,0.34
8,3,32-16-8,0.0001,0.34,0.34,0.34,0.34
9,3,32-32-32,0.01,0.67,0.68,0.60,0.60


# Выводы

При использовании предобученных моделей для признаков результаты нейронных сетей получились хуже, скорее всего это связано с тем, что обучение шло на полных словах, без опечаток, а представленный набор данных содержит некоторое количество твитов, где часто встречаются дефектные слова. Но последнее улучшит результат и для самостоятельно обученных моделей признаков. В сводной таблице можно заметить увеличение количество застревания метода градиентного спуска в локальных минимумах, особенно при увеличении количества скрытых слоев.